In [1]:
import numpy as np
import pickle
from time import time

from sph_lib.ops import MainClass

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
dim         = 3
methods     = ['ngp', 'cic', 'tsc']
dataset     = 'cosmo'
gridnum     = 128
verbose     = False
use_openmp  = True
num_open_mp_threads = [None, 1, 2, 4, 8, 16, 32]

for method in methods:
    print("-" * 50)
    print(f"Method: {method} {dim}D")
    
    with open(f'../../../data/dataset_{dataset}_{dim}d.pkl', 'rb') as f:
        data = pickle.load(f)
    pos = data['pos'][:]
    mass = data['mass'][:, np.newaxis]
    boxsize = data['boxsize']

        
    # initialize MainClass
    sim = MainClass(
                    positions=pos,
                    masses=mass,
                    boxsize=boxsize,
                    verbose=verbose,
                    )
    
    # no need to compute hsm for this test

    timings = []
    for num_threads in num_open_mp_threads:
        
        start = time()
        res = sim.deposit_to_grid(
            fields=mass,
            averaged=[False] * mass.shape[1],
            gridnums=gridnum,
            method=method,
            use_openmp=use_openmp,
            omp_threads=num_threads,
            )
        end = time()
        timings.append(end - start)
        print(f"Test on {dataset}:\t{timings[0] / timings[-1]:.2f}x speedup with {num_threads} threads")

--------------------------------------------------
Method: ngp 3D
Test on cosmo:	1.00x speedup with None threads
Test on cosmo:	0.85x speedup with 1 threads
Test on cosmo:	0.97x speedup with 2 threads
Test on cosmo:	1.04x speedup with 4 threads
Test on cosmo:	0.95x speedup with 8 threads
Test on cosmo:	1.05x speedup with 16 threads
Test on cosmo:	0.71x speedup with 32 threads
--------------------------------------------------
Method: cic 3D
Test on cosmo:	1.00x speedup with None threads
Test on cosmo:	0.46x speedup with 1 threads
Test on cosmo:	0.77x speedup with 2 threads
Test on cosmo:	0.92x speedup with 4 threads
Test on cosmo:	1.01x speedup with 8 threads
Test on cosmo:	0.95x speedup with 16 threads
Test on cosmo:	1.02x speedup with 32 threads
--------------------------------------------------
Method: tsc 3D
Test on cosmo:	1.00x speedup with None threads
Test on cosmo:	0.33x speedup with 1 threads
Test on cosmo:	0.55x speedup with 2 threads
Test on cosmo:	0.84x speedup with 4 threa

In [3]:
methods     = ['isotropic', 'anisotropic']
num_nn      = 64

for method in methods:
    print("-" * 50)
    print(f"Method: {method} {dim}D")
    
    with open(f'../../../data/dataset_{dataset}_{dim}d.pkl', 'rb') as f:
        data = pickle.load(f)
    pos = data['pos'][:]
    mass = data['mass'][:, np.newaxis]
    boxsize = data['boxsize']

    # initialize MainClass
    sim = MainClass(
                    positions=pos,
                    masses=mass,
                    boxsize=boxsize,
                    verbose=verbose,
                    )
    
    # need to compute hsm for this test
    sim.compute_smoothing_lengths(num_neighbors=num_nn, mode=method)

    timings = []
    for num_threads in num_open_mp_threads:
        
        start = time()
        res = sim.deposit_to_grid(
            fields=mass,
            averaged=[False] * mass.shape[1],
            gridnums=gridnum,
            method=method,
            use_openmp=use_openmp,
            omp_threads=num_threads,
            )
        end = time()
        timings.append(end - start)
        print(f"Test on {dataset}:\t{timings[0] / timings[-1]:.2f}x speedup with {num_threads} threads")

--------------------------------------------------
Method: isotropic 3D
Test on cosmo:	1.00x speedup with None threads
Test on cosmo:	0.19x speedup with 1 threads
Test on cosmo:	0.36x speedup with 2 threads
Test on cosmo:	0.69x speedup with 4 threads
Test on cosmo:	0.94x speedup with 8 threads
Test on cosmo:	1.05x speedup with 16 threads
Test on cosmo:	0.96x speedup with 32 threads
--------------------------------------------------
Method: anisotropic 3D
Test on cosmo:	1.00x speedup with None threads
Test on cosmo:	0.22x speedup with 1 threads
Test on cosmo:	0.43x speedup with 2 threads
Test on cosmo:	0.83x speedup with 4 threads
Test on cosmo:	1.11x speedup with 8 threads
Test on cosmo:	1.26x speedup with 16 threads
Test on cosmo:	1.24x speedup with 32 threads
